## 顯示 OSM 地圖 (folium)

In [4]:
from geopy.geocoders import Nominatim
import folium
from folium.plugins import LocateControl  # 引入 LocateControl


In [ ]:
# 初始化地理編碼器
geolocator = Nominatim(user_agent="geo_test_app")

# 查詢起始地點: 國立臺灣師範大學
start_location = geolocator.geocode("國立臺灣師範大學")

# 檢查是否找到地點
if start_location:
    print(f"起始地點: {start_location.address}")
    print(f"經緯度: ({start_location.latitude}, {start_location.longitude})")

    # 將地圖以起始地點為中心建立
    map_osm = folium.Map(
        location=[start_location.latitude, start_location.longitude],
        zoom_start=15
    )

    # 在地圖上標示起始地點
    folium.Marker(
        [start_location.latitude, start_location.longitude],
        popup="國立臺灣師範大學",
        tooltip="起始地點"
    ).add_to(map_osm)

    # 新增定位目前位置的按鈕
    LocateControl().add_to(map_osm)

    # 儲存地圖為 HTML 檔案
    map_file = "ntnu.html"
    map_osm.save(map_file)
    print(f"地圖已儲存為 {map_file}，請用瀏覽器開啟查看。")
else:
    print("無法找到地點，請確認地址。")


### 寵物友善餐廳

In [6]:
import json
import pandas as pd

In [9]:
# 讀取 JSON 資料
file_path = "pet_friendly_restaurants.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# 初始化地理編碼器
geolocator = Nominatim(user_agent="geo_test_app")

# 儲存地址與經緯度結果
locations = []

# 查詢每個地址的經緯度
for item in data:
    name = item["Restaurant Name"]
    address = item["Address"]
    try:
        # 使用地址進行地理查詢
        location = geolocator.geocode(address)
        if location:
            locations.append({
                "name": name,
                "address": address,
                "latitude": location.latitude,
                "longitude": location.longitude
            })
            print(f"查詢成功: {name} ({address}) -> ({location.latitude}, {location.longitude})")
        else:
            print(f"查詢失敗: {name} ({address})")
            locations.append({"name": name, "address": address, "latitude": None, "longitude": None})
    except Exception as e:
        print(f"查詢時出現錯誤: {name} ({address}), 錯誤訊息: {e}")
        locations.append({"name": name, "address": address, "latitude": None, "longitude": None})

# 將結果保存到 DataFrame
locations_df = pd.DataFrame(locations)

# 儲存結果為 CSV 檔案
output_csv = "pet_friendly_restaurants_coordinates.csv"
locations_df.to_csv(output_csv, index=False, encoding="utf-8")
print(f"地址經緯度資料已保存到 {output_csv}")


查詢成功: 賈福廚房 JFULL (台北市大安區復興南路二段) -> (25.03194895, 121.5421290667586)
查詢成功: 光一肆號 (台北市大安區新生南路三段) -> (25.0233477, 121.53530431351457)
查詢成功: Toast Chat (台北市大安區光復南路) -> (25.04320294998675, 121.5576586083291)
查詢成功: Become早午餐 (台北市大安區信義路四段) -> (25.033105958874025, 121.55339420804869)
查詢成功: 貳樓咖啡（敦南店） (台北市大安區敦化南路二段) -> (25.0333251, 121.54855505)
查詢成功: 小春日和 (台北市大安區信義路三段) -> (25.033355738654592, 121.54068749946612)
查詢成功: Mianto 艾果豐 (台北市大安區仁愛路) -> (25.038599, 121.535631)
查詢失敗: PS Bubu 金屋藏車食堂 (台北市大安區溫州街26號)
查詢成功: Luigi 路易奇電力公司 (台北市大安區復興南路一段) -> (25.04333128436147, 121.54389343580793)
查詢成功: 舒服生活 Truffles Living (台北市大安區文昌街) -> (25.032800438292195, 121.55077794534427)
查詢成功: 兔 Dreams 寵物餐廳 (台北市大安區新生南路一段) -> (25.04231625, 121.53291585)
查詢成功: 慵懶咖啡 Lazzy Cafe (台北市大安區新生南路二段) -> (25.029748599999998, 121.53272466713736)
查詢成功: The Antipodean (台北市大安區復興南路一段) -> (25.04333128436147, 121.54389343580793)
地址經緯度資料已保存到 pet_friendly_restaurants_coordinates.csv


In [11]:

# 繪製地圖
if not locations_df.empty:
    # 建立地圖，中心為所有地址平均座標
    valid_locations = locations_df.dropna(subset=["latitude", "longitude"])
    if not valid_locations.empty:
        center_lat = valid_locations["latitude"].mean()
        center_lon = valid_locations["longitude"].mean()
        map_osm = folium.Map(location=[center_lat, center_lon], zoom_start=14)

        # 將所有地址標記到地圖
        for _, row in valid_locations.iterrows():
            folium.Marker(
                location=[row["latitude"], row["longitude"]],
                popup=row["name"],
                tooltip=row["address"]
            ).add_to(map_osm)

        # 新增「定位目前位置」按鈕
        LocateControl().add_to(map_osm)

        # 儲存地圖為 HTML 檔案
        map_html = "pet_friendly_restaurants_map.html"
        map_osm.save(map_html)
        print(f"地圖已保存為 {map_html}，請用瀏覽器開啟查看。")
    else:
        print("無有效地點資料，無法生成地圖。")
else:
    print("未能生成有效的地址資料，請檢查輸入檔案。")


地圖已保存為 pet_friendly_restaurants_map.html，請用瀏覽器開啟查看。
